# Day 15

In [424]:
class Actor:
    def __init__(self, grid, x, y):
        self.grid = grid
        self.x = x
        self.y = y

    def __str__(self):
        return '{0}({1.x}, {1.y})'.format(type(self).__name__, self)

    @staticmethod
    def fromcode(code, grid, x, y):
        if code == 'E':
            return Elf(grid, x, y)
        elif code == 'G':
            return Goblin(grid, x, y)

    def go(self):
        pass
    
    def move(self):
        pass

    @property
    def reachable(self):
        todo = [(self.x + 1, self.y), (self.x - 1, self.y), (self.x, self.y + 1), (self.x, self.y - 1)]
        visited = set()
        while todo:
            pt = todo.pop()
            if pt in visited:
                continue
            visited.add(pt)
            x, y = pt
            if self.grid[y][x] == '.':
                todo.extend([(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)])
                yield pt


    __repr__ = __str__


class Goblin(Actor):
    pass


class Elf(Actor):
    pass

In [425]:
def parse(f):
    grid = [list(line.rstrip()) for line in f]

    actors = []
    for y, row in enumerate(grid):
        for x, block in enumerate(row):
            if block in ('E', 'G'):
                actors.append(Actor.fromcode(block, grid, x, y))

    return actors, grid

In [426]:
from io import StringIO
f = StringIO('''#######
#E..G.#
#...#.#
#.G.#G#
#######''')

In [427]:
actors, grid = parse(f)

In [428]:
actors

[Elf(1, 1), Goblin(4, 1), Goblin(2, 3), Goblin(5, 3)]

In [429]:
set(actors[0].reachable)

{(1, 2), (1, 3), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3)}

# Day 17

In [157]:
import numpy as np

def parse(f):
    grid = np.zeros((2000, 2000), dtype=np.int32)
    for line in f:
        fixed, vein = line.split(',')
        axis_fixed, val = fixed.split('=')
        axis_vein, rng = vein.split('=')
        start, end = map(int, rng.split('..'))
        if axis_fixed == 'x':
            grid[start:end + 1, int(val)] = 1
        else:
            grid[int(val), start:end + 1] = 1
    return grid

In [158]:
from io import StringIO
f = StringIO('''x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504''')
grid = parse(f)

In [162]:
grid[0:, 500].nonzero?

Object `nonzero` not found.


In [ ]:
grid[0:, 500].nonzero

In [ ]:
def flow(grid, source=(0, 500)):
    sources = [source]
    while sources:
        src = sources.pop()

# Day 22

In [116]:
from functools import lru_cache

@lru_cache(100000)
def risk_level(x, y, depth, target_loc):
    return erosion_level(x, y, depth, target_loc) % 3

@lru_cache(100000)
def erosion_level(x, y, depth, target_loc):
    return (geologic_index(x, y, depth, target_loc) + depth) % 20183

@lru_cache(100000)
def geologic_index(x, y, depth, target_loc):
    if (x, y) == target_loc:
        return 0
    if x == 0:
        return y * 48271
    if y == 0:
        return x * 16807
    return erosion_level(x - 1, y, depth, target_loc) * erosion_level(x, y - 1, depth, target_loc)

In [117]:
assert risk_level(0, 0, 510, (10, 10)) == 0
assert risk_level(1, 0, 510, (10, 10)) == 1
assert risk_level(0, 1, 510, (10, 10)) == 0
assert risk_level(1, 1, 510, (10, 10)) == 2
assert risk_level(10, 10, 510, (10, 10)) == 0

In [118]:
def total_risk(depth, target):
    return sum(risk_level(i, j, depth, target) for i in range(target[0] + 1) for j in range(target[1] + 1))

In [119]:
target = (10, 10)
depth = 510
assert total_risk(depth, target) == 114

In [120]:
target = (13, 704)
depth = 9465
print('Part 1:', total_risk(depth, target))

Part 1: 9940
